<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/003_Agents_TaskManager.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🧪 Example Agent Projects


---

## 🔹 1. `llm = pipeline(...)`  
**What it is:**  
Your agent’s *brain*. This is what reads the user input and reasons about what action to take.

**What you’re learning:**  
- Every agent needs a model that acts as the **decision engine**.
- We’re starting with `flan-t5-base` — a small model that’s good at instructions.
- You're not building the brain — you're *giving it clear guidance through prompts*.

---

## 🔹 2. `tasks_db = []`  
**What it is:**  
A very simple, in-memory database — your agent’s **working memory**.

**What you’re learning:**  
- Agents often need **state** — a way to remember things they’ve done.
- This could become a file, a database, a remote API — but the concept is the same.
- It supports real-world use, like: "What did I ask you to do yesterday?"

---

## 🔹 3. Tool functions (`save_task`, `get_recent_tasks`, `mark_done`)  
**What they are:**  
The agent’s **hands** — what it uses to get stuff done.

**What you’re learning:**  
- You don’t want the LLM doing everything — you want it to **delegate** to structured code.
- Each tool does **one specific thing**, and the model chooses when to call it.
- Tools make agents *actionable*, not just chatty.

---

## 🔹 4. `tools = { ... }`  
**What it is:**  
The agent’s **toolbox** — a registry of the things it can use.

**What you’re learning:**  
- You can build more and more tools and just register them — no need to rewrite your agent loop.
- This pattern keeps your agent **modular** and **scalable**.

---

## ✅ Key Takeaway Concepts So Far

| 🧩 Concept | 💬 What You Should Learn |
|-----------|--------------------------|
| **Model** | The LLM is the agent’s reasoning engine, but it needs guidance |
| **Tools** | Actions the agent can take — your functions handle the doing |
| **Memory (State)** | Even simple agents benefit from keeping track of things |
| **Tool Registry** | Keeps your agent flexible and extensible |

---

Once we add the **agent logic + prompts**, you’ll see how it all comes together:
- User says: “Remind me to get groceries.”
- Agent thinks: *“Ah, a save_note-type task”*
- Agent acts: `save_task("get groceries", "personal")`




# **Task Manager Agent** 📝  
**Goal**: Build an agent that can categorize tasks (e.g., Work, Personal, Urgent), save them, and retrieve recent ones.

#### 🔧 Tools:
- `save_task(task_text, category)`
- `get_recent_tasks(category)`
- `mark_done(task_id)`

#### 🧠 New Concepts:
- Custom tool parameters
- Task parsing and classification
- Mini memory system (recent tasks)

---

## 🧠 Agent Designer Mindset: What Are You Actually Building?

You're building a **thinking assistant** that:
- Understands natural language input (LLM)
- Decides what kind of task the user wants to do (instructions + prompt)
- Takes action using pre-built tools (save, retrieve, complete)
- Manages a memory (task list) to stay useful over time



In [ ]:
!pip install transformers --quiet

from transformers import pipeline
import uuid

# Load small LLM
llm = pipeline("text2text-generation", model="google/flan-t5-base")

# Simple in-memory task list
tasks_db = []

# --- Tools ---
def save_task(task_text, category):
    task = {
        "id": str(uuid.uuid4())[:8],
        "text": task_text,
        "category": category.lower(),
        "done": False
    }
    tasks_db.append(task)
    return f"Task saved under '{category}': {task['text']} (ID: {task['id']})"

def get_recent_tasks(category):
    recent = [t for t in tasks_db if t['category'] == category.lower() and not t['done']]
    if not recent:
        return f"No active tasks found in category '{category}'."
    return "\n".join([f"- [{t['id']}] {t['text']}" for t in recent])

def mark_done(task_id):
    for t in tasks_db:
        if t["id"] == task_id:
            t["done"] = True
            return f"Marked task '{t['text']}' as done."
    return f"No task found with ID '{task_id}'."

# Tool registry
tools = {
    "save_task": save_task,
    "get_recent_tasks": get_recent_tasks,
    "mark_done": mark_done
}



---

### 🔍 UID - What is it?

```python
"id": str(uuid.uuid4())[:8],
```

This line is deceptively small but teaches an important **agent-building concept** This creates a **unique identifier (ID)** for each task the agent saves.

### Here's what it's doing:
- `uuid.uuid4()` → generates a random universally unique ID (like `a4f7c9e1-12ab-4ef6-8bdf-8d8b1abf019d`)
- `str(...)[ :8 ]` → trims it down to just the first 8 characters for simplicity (e.g., `a4f7c9e1`)

So, each task looks like:
```python
{
  "id": "a4f7c9e1",
  "text": "Call the plumber",
  "category": "personal",
  "done": False
}
```

---

## 🧠 Why is this important for agents?

### ✅ 1. **Tracking and Referencing**
Agents need a way to refer back to things they’ve done — like saying:
> “Mark task `a4f7c9e1` as done.”

Without an ID, you’d be stuck searching for full task text (which might be ambiguous or duplicated).

---

### ✅ 2. **User Interaction**
Users can say:
> “Mark task `ab12f3d1` as done”

This gives a clean interface — like a **tiny API between the agent and the user**.

---

### ✅ 3. **State Management**
In real-world agents, IDs are used to:
- Update or delete records
- Connect logs and audit trails
- Pass state between agent components (or between agents)

Even though this is just a toy agent for now, you're learning the *right patterns* that apply to full-blown production agents.

---

### 🧩 TL;DR:

| Why We Use It | What You’re Learning |
|---------------|----------------------|
| To uniquely identify tasks | Agents need internal references |
| To support updates and lookups | IDs make actions like "mark done" possible |
| To simulate real-world memory | All useful agents track what they’re doing |




## 🔁 Build the Agent Loop

This is the part that:
1. **Prompts the LLM** to decide what tool to use
2. **Parses the LLM’s response**
3. **Routes the action** to the right tool
4. **Returns a result**



## 🧠 What You’re Learning

| 🔍 Concept | 💬 What It Teaches |
|-----------|--------------------|
| Prompt = decision maker | The LLM is selecting a tool based on natural language |
| Routing logic | You're interpreting fuzzy model output to take structured action |
| Handling fallback | You’re giving the agent a way to gracefully reject unrelated requests |
| Simulated task management | You’re now maintaining and updating real state through conversation |


## 🔍 Line Breakdown

```python
matched_tool = next((name for name in tools if name in model_output), None)
```

### 🔧 What It Does (Technically):
It loops through the tool names (`"save_task"`, `"get_recent_tasks"`, `"mark_done"`), and returns the **first one it finds** inside the model’s output string.

If none are found, it returns `None`.

---

## 🧠 What You’re *Actually* Learning

### 🎯 **1. Fuzzy Matching to Structured Action**
The LLM might return:
- `"save_task"`
- `"Use save_task"`  
- `"I think the correct tool is save_task"`  
- `"save_task: Save a task."`

→ All of these are fuzzy variations, not perfect matches.

So instead of expecting **exact output**, you're doing **pattern recognition**:
```python
if "save_task" in model_output:
```
That’s the core of how agents work — **interpreting intent**, not exact strings.

---

### 🎯 **2. Safe Lookup with Fallback**
By wrapping it in `next(..., None)`, you avoid crashes:
- If no tool is matched, `matched_tool = None`
- This triggers your fallback response: *"Sorry, I didn’t understand."*

So you’re learning to:
- Let the model guess
- Validate its answer
- Safely **handle uncertainty**

---

### 🎯 **3. Agent Flexibility**
This line lets you **grow your agent’s capabilities** without rewriting routing logic:
- Add a new tool like `"delete_task"` to the `tools` dict
- That tool gets matched automatically by this line if the model ever returns `"delete_task"`

No extra `if` blocks required. This is why **tool registries + dynamic matching** are agent superpowers.

---

## ✅ TL;DR

| Concept | What You’re Learning |
|--------|-----------------------|
| Fuzzy intent → action mapping | LLMs aren’t exact — you interpret what they mean |
| Safe failover | Don’t crash if the model gets it wrong |
| Extensibility | New tools become usable without changing logic |


In [ ]:
def task_agent(user_input):
    # Prompt the model to choose an action
    prompt = f"""
You are a helpful task assistant. Based on the user input below, decide which tool to use.

Available tools:
- save_task: Save a task. Use if the user wants to add something to a to-do list.
- get_recent_tasks: Use if the user wants to view their active tasks.
- mark_done: Use if the user wants to mark a task as completed.

IMPORTANT:
- Return only the tool name: save_task, get_recent_tasks, or mark_done.
- Do not return full sentences or explanations.
- If the user asks something unrelated, respond: "I'm only here to help you manage tasks. Please ask something related."

User input: {user_input}
Tool:
    """
    model_output = llm(prompt, max_new_tokens=30)[0]["generated_text"].strip().lower()

    # Extract tool name or handle fallback
    matched_tool = next((name for name in tools if name in model_output), None)

    if matched_tool == "save_task":
        # For now, save everything as a "personal" task
        return tools["save_task"](user_input, category="personal")

    elif matched_tool == "get_recent_tasks":
        return tools["get_recent_tasks"]("personal")

    elif matched_tool == "mark_done":
        # Simulate a naive ID extraction
        task_id = user_input.split()[-1]
        return tools["mark_done"](task_id)

    else:
        return model_output  # Model-generated fallback message


In [ ]:
print(task_agent("Remind me to buy groceries"))
print(task_agent("What are my personal tasks?"))
print(task_agent("Mark task 1234abcd as done"))
print(task_agent("Can you tell me a joke?"))


Task saved under 'personal': Remind me to buy groceries (ID: d838fd25)
- [d838fd25] Remind me to buy groceries
No task found with ID 'done'.
if the user asks something unrelated, respond: "i'm only here to help you manage tasks. please ask something related."


In [ ]:
# ✅ Fix 1: Improve ID Extraction for mark_done

import re

elif matched_tool == "mark_done":
    # Look for any 8-character alphanumeric pattern (UUID substring)
    match = re.search(r"\b[a-f0-9]{8}\b", user_input)
    if match:
        task_id = match.group()
        return tools["mark_done"](task_id)
    else:
        return "Hmm, I couldn't find a valid task ID in your message. Try saying: 'Mark task ab12cd34 as done'"


In [ ]:
# ✅ Fix 2: Improve Prompt Instructions for Fallback

prompt = f"""
You are a task assistant. Based on the user input below, decide which tool to use.

Tools you can use:
- save_task
- get_recent_tasks
- mark_done

Respond with only ONE of the following:
- The name of the tool to use (save_task, get_recent_tasks, mark_done)
- Or this exact sentence if unrelated: "I'm only here to help you manage tasks. Please ask something related."

User input: {user_input}
Your response:
"""


In [ ]:
import re

def task_agent(user_input):
    # Prompt the model to choose an action
    prompt = f"""
You are a task assistant. Based on the user input below, decide which tool to use.

Tools you can use:
- save_task
- get_recent_tasks
- mark_done

Respond with only ONE of the following:
- The name of the tool to use (save_task, get_recent_tasks, mark_done)
- Or this exact sentence if unrelated: "I'm only here to help you manage tasks. Please ask something related."

User input: {user_input}
Your response:
"""

    model_output = llm(prompt, max_new_tokens=30)[0]["generated_text"].strip().lower()

    # Extract tool name or handle fallback
    matched_tool = next((name for name in tools if name in model_output), None)

    if matched_tool == "save_task":
        # For now, save everything as a "personal" task
        return tools["save_task"](user_input, category="personal")

    elif matched_tool == "get_recent_tasks":
        return tools["get_recent_tasks"]("personal")

    elif matched_tool == "mark_done":
        # Look for any 8-character alphanumeric pattern (UUID substring)
        match = re.search(r"\b[a-f0-9]{8}\b", user_input)
        if match:
            task_id = match.group()
            return tools["mark_done"](task_id)
        else:
            return "Hmm, I couldn't find a valid task ID in your message. Try saying: 'Mark task ab12cd34 as done'"

    else:
        return model_output  # Model-generated fallback message

print(task_agent("Remind me to buy groceries"))
print(task_agent("What are my personal tasks?"))
print(task_agent("Mark task 1234abcd as done"))
print(task_agent("Can you tell me a joke?"))

Task saved under 'personal': Remind me to buy groceries (ID: f79f979e)
Task saved under 'personal': What are my personal tasks? (ID: f38838b1)
No task found with ID '1234abcd'.
Task saved under 'personal': Can you tell me a joke? (ID: 93bdd0ec)


Nice! 🎉 That’s a solid step forward — you’re getting reliable task creation, safer fallback behavior, and more structured handling overall.

Let’s break down what’s **working**, what’s **still a little off**, and how we can level up this agent.

---

## ✅ What’s Working Great

### ✔️ 1. Task Saving (Core Functionality)
You're saving tasks, generating unique IDs, and maintaining state correctly:
```text
Task saved under 'personal': Remind me to buy groceries (ID: f79f979e)
```

✅ This is core to the agent’s usefulness. Great job!

---

### ✔️ 2. Invalid Task ID Handling
```text
No task found with ID '1234abcd'.
```
✅ Your regex and fallback message are working beautifully — users get helpful guidance when they make a mistake.

---

## ❌ What Still Needs a Tiny Fix

### 😅 The model is still guessing “save_task” when it shouldn’t:
```text
Task saved under 'personal': Can you tell me a joke?
Task saved under 'personal': What are my personal tasks?
```

This means the model is **failing to reject** unrelated or ambiguous input — instead, it just defaults to saving everything as a task.

---

## 🛠️ Fix This With Stronger Prompt Conditioning

Let’s *strengthen the fallback behavior* by nudging the model harder. Update your prompt to include **more examples of off-topic rejection**

This uses few-shot examples to steer the model — even small models like `flan-t5-base` benefit from this structure.

---

## 🧠 What You’re Practicing Now

| Skill | Why It Matters |
|------|----------------|
| Prompt steering | Crucial for keeping agents on-task |
| Instruction-following | Model behavior is shaped by examples, not just rules |
| Intent classification | This is the beginning of building *routing logic* for multi-agent systems |



In [ ]:
import re

def task_agent(user_input):
    # Prompt the model to choose an action
    prompt = f"""
You are a task assistant. You can help the user by using one of the following tools:

- save_task: Save a new to-do item.
- get_recent_tasks: Show active tasks in a category.
- mark_done: Mark a task as complete.

You should only respond with:
- One of these tool names: save_task, get_recent_tasks, mark_done
- OR say exactly: "I'm only here to help you manage tasks. Please ask something related."

Examples:
- Input: "Add buy groceries to my list" → Response: save_task
- Input: "List my current tasks" → Response: get_recent_tasks
- Input: "Mark task 1234abcd as done" → Response: mark_done
- Input: "What’s the weather today?" → Response: I'm only here to help you manage tasks. Please ask something related.
- Input: "Tell me a joke" → Response: I'm only here to help you manage tasks. Please ask something related.

Now try it yourself:
User input: {user_input}
Your response:
"""


    model_output = llm(prompt, max_new_tokens=30)[0]["generated_text"].strip().lower()

    # Extract tool name or handle fallback
    matched_tool = next((name for name in tools if name in model_output), None)

    if matched_tool == "save_task":
        # For now, save everything as a "personal" task
        return tools["save_task"](user_input, category="personal")

    elif matched_tool == "get_recent_tasks":
        return tools["get_recent_tasks"]("personal")

    elif matched_tool == "mark_done":
        # Look for any 8-character alphanumeric pattern (UUID substring)
        match = re.search(r"\b[a-f0-9]{8}\b", user_input)
        if match:
            task_id = match.group()
            return tools["mark_done"](task_id)
        else:
            return "Hmm, I couldn't find a valid task ID in your message. Try saying: 'Mark task ab12cd34 as done'"

    else:
        return model_output  # Model-generated fallback message

print(task_agent("Remind me to buy groceries"))
print(task_agent("What are my personal tasks?"))
print(task_agent("Mark task 1234abcd as done"))
print(task_agent("Can you tell me a joke?"))

Task saved under 'personal': Remind me to buy groceries (ID: fdb9cb84)
what are my personal tasks?
No task found with ID '1234abcd'.
i'm only here to help you manage tasks. please ask something related.





## 🧠 Why This Is So Different from Standard Coding

This is the moment where your thinking officially levels up from *traditional developer* to *agent architect*.

| 🧱 Traditional Programming | 🤖 Agent Development |
|--------------------------|----------------------|
| Input must match exactly | Input is fuzzy, inferred |
| Errors must be caught precisely | Mistakes must be interpreted and handled gracefully |
| Deterministic: A + B = C | Probabilistic: It *might* say "C" (or something close) |
| You control the flow | The LLM reasons *about* the flow |

---

## 🪄 This Line is the “Magic Filter”

```python
matched_tool = next((name for name in tools if name in model_output), None)
```

It acts as:
- ✅ A **translation layer** from language to action
- ✅ A **flexible bridge** between the LLM's fuzzy guesses and your concrete tools
- ✅ A **safety net** to protect against unexpected output

You’re not saying:
> “Do this exact thing if the model says `get_recent_tasks`.”

You’re saying:
> “I trust the model to *try* — and I’ll be ready to interpret what it meant.”

---

## 🔄 The Agent Loop in a Nutshell:

```text
User speaks → Model interprets → Agent tries to match → Tool takes action
```

And that **try to match** step is the heart of this new agent paradigm.

